In [1]:
# import functions
import pandas as pd
import numpy as np
import codecs
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler                 #normalising features
from sklearn.cross_validation import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.pipeline import make_pipeline
from sklearn.cross_validation import KFold

from sklearn.linear_model import RidgeClassifier
from sklearn.linear_model import Lasso
from sklearn.svm import SVC, SVR

from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier

from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor
from sknn import mlp

from scipy.optimize import minimize

import theano
import theano.tensor as T

%matplotlib inline

np.random.seed=10


In [34]:
import matplotlib.pyplot as plt

from py4j.java_gateway import JavaGateway

In [35]:
%matplotlib inline

# set up connection

In [36]:
gw = JavaGateway() # New gateway connection
bridge = gw.entry_point

# load netlogo model

In [37]:
# Path to the model:
models_path = "D:/Work/PhD/Github/NetLogoModel/"
model_name = "PhD_InitialModelV2.nlogo"

In [38]:
modelLocation = models_path + model_name

In [44]:
bridge.openModel(modelLocation)

ERROR:py4j.java_gateway:Error while sending or receiving.
Traceback (most recent call last):
  File "C:\Users\Yanjie\Anaconda\lib\site-packages\py4j\java_gateway.py", line 761, in send_command
    self.socket.sendall(command.encode("utf-8"))
  File "C:\Users\Yanjie\Anaconda\lib\socket.py", line 228, in meth
    return getattr(self._sock,name)(*args)
error: [Errno 10054] An existing connection was forcibly closed by the remote host


# Run one iteration of the model, one command at a time:

In [45]:
bridge.command("set grid-size 21")
bridge.command("set nb-people 1")
bridge.command("set Show_Names_Nodes? False")
bridge.command("set Show_Names_people? False")
bridge.command("set car-speed 1")
bridge.command("set mobiletowerdata? True")
bridge.command("set socialmedia_data? True")
bridge.command("set grids_covered_vector 5")
bridge.command("set avg_num_calls_perday 5")
bridge.command("set avg_call_duration_mins 2")

bridge.command("random-seed 0")
bridge.command("setup")
bridge.command("repeat 1440 [go]")

In [46]:
bridge.command("repeat 1440 [go]")

# Have the model report results

In [2]:
!head -n 15 ./agentdata.txt

 0 0 0 441 15 0 30 -37 453
 1 0 1 441 15 0 30 -37 453
 2 0 2 441 15 0 30 -37 453
 3 0 3 441 15 0 30 -37 453
 4 0 4 441 15 0 30 -37 453
 5 0 5 441 15 0 30 -37 453
 6 0 6 441 15 0 30 -37 453
 7 0 7 441 15 0 30 -37 453
 8 0 8 441 15 0 30 -37 453
 9 0 9 441 15 0 30 -37 453
 10 0 10 441 15 0 30 -37 453
 11 0 11 441 15 0 30 -37 453
 12 0 12 441 15 0 30 -37 453
 13 0 13 441 15 0 30 -37 453
 14 0 14 441 15 0 30 -37 453


In [3]:
!head -n 15 ./mobiletowers.txt

 406 0 406 441 0 0 15 9.523809523809524 453 32 -32
 560 0 560 441 0 1 25 9.523809523809524 465 0 16
 842 0 842 441 0 1 35 9.523809523809524 454 32 16
 1028 0 1028 441 0 1 25 9.523809523809524 470 -32 16
 1902 1 462 441 0 0 15 9.523809523809524 453 32 -32
 2298 1 858 441 0 1 35 9.523809523809524 460 16 16
 2306 1 866 441 0 1 35 9.523809523809524 460 16 16
 2455 1 1015 441 0 0 25 9.523809523809524 470 -32 16
 2584 1 1144 441 0 1 15 9.523809523809524 457 0 -16
 3073 2 193 441 0 0 15 9.523809523809524 453 32 -32
 3434 2 554 441 0 1 25 9.523809523809524 452 0 0
 3442 2 562 441 0 1 25 9.523809523809524 465 0 16
 3899 2 1019 441 0 0 25 9.523809523809524 470 -32 16
 4042 2 1162 441 0 1 15 9.523809523809524 447 16 -16
 4149 2 1269 441 0 0 15 9.523809523809524 453 32 -32


In [24]:
# import simuated data
def rawimport(path, columnname):
    agentdata = pd.read_csv(path, delimiter=' ', header=None, names=columnname)
    agentdata = agentdata.drop(labels="todrop", axis=1)
    return agentdata
#mobiledata = pd.read_csv('./mobiletowers.txt', delimiter=' ', header=None, names=["todrop", "acc_time", "day", "minute", "pid", 
#                                                                                  "tpid", "zone", "radius", "tid", "xcor", "ycor"] )
#aa.drop(labels="todrop", axis=1)


In [25]:
path = './agentdata.txt'
columnname = ["todrop", "acc_time", "day", "minute", "pid", "zone", "status", "xcor", "ycor", "tid"]
agentdata = rawimport(path, columnname)


In [26]:
path = './mobiletowers.txt'
columnname = ["todrop", "acc_time", "day", "minute", "pid", "tpid", "zone", "radius", "tid", "xcor", "ycor"]
mobiledata = rawimport(path, columnname)


In [47]:
mobiledata_cp = agentdata.iloc[:,0:3]
mobiledata_cp

In [49]:
mobiledata_cp = pd.concat([agentdata, mobiledata], axis=1, join='outer', join_axes=None, ignore_index=False,
       keys=None, levels=None, names=None, verify_integrity=False)
#check documentation

In [50]:
mobiledata_cp

acc_time  day  minute  pid  zone  status       xcor       ycor  tid  \
0             0    0       0  441    15       0  30.000000 -37.000000  453   
1             1    0       1  441    15       0  30.000000 -37.000000  453   
2             2    0       2  441    15       0  30.000000 -37.000000  453   
3             3    0       3  441    15       0  30.000000 -37.000000  453   
4             4    0       4  441    15       0  30.000000 -37.000000  453   
5             5    0       5  441    15       0  30.000000 -37.000000  453   
6             6    0       6  441    15       0  30.000000 -37.000000  453   
7             7    0       7  441    15       0  30.000000 -37.000000  453   
8             8    0       8  441    15       0  30.000000 -37.000000  453   
9             9    0       9  441    15       0  30.000000 -37.000000  453   
10           10    0      10  441    15       0  30.000000 -37.000000  453   
11           11    0      11  441    15       0  30.000000 -37.000000  453   
12           12    0      12  441    15       0  30.000000 -37.000000  453   
13           13    0      13  441    15       0  30.000000 -37.000000  453   
14           14    0      14  441    15       0  30.000000 -37.000000  453   
15           15    0      15  441    15       0  30.000000 -37.000000  453   
16           16    0      16  441    15       0  30.000000 -37.000000  453   
17           17    0      17  441    15       0  30.000000 -37.000000  453   
18           18    0      18  441    15       0  30.000000 -37.000000  453   
19           19    0      19  441    15       0  30.000000 -37.000000  453   
20           20    0      20  441    15       0  30.000000 -37.000000  453   
21           21    0      21  441    15       0  30.000000 -37.000000  453   
22           22    0      22  441    15       0  30.000000 -37.000000  453   
23           23    0      23  441    15       0  30.000000 -37.000000  453   
24           24    0      24  441    15       0  30.000000 -37.000000  453   
25           25    0      25  441    15       0  30.000000 -37.000000  453   
26           26    0      26  441    15       0  30.000000 -37.000000  453   
27           27    0      27  441    15       0  30.000000 -37.000000  453   
28           28    0      28  441    15       0  30.000000 -37.000000  453   
29           29    0      29  441    15       0  30.000000 -37.000000  453   
...         ...  ...     ...  ...   ...     ...        ...        ...  ...   
14370     14370    9    1410  441    15       0  29.598127 -37.200936  453   
14371     14371    9    1411  441    15       0  29.598127 -37.200936  453   
14372     14372    9    1412  441    15       0  29.598127 -37.200936  453   
14373     14373    9    1413  441    15       0  29.598127 -37.200936  453   
14374     14374    9    1414  441    15       0  29.598127 -37.200936  453   
14375     14375    9    1415  441    15       0  29.598127 -37.200936  453   
14376     14376    9    1416  441    15       0  29.598127 -37.200936  453   
14377     14377    9    1417  441    15       0  29.598127 -37.200936  453   
14378     14378    9    1418  441    15       0  29.598127 -37.200936  453   
14379     14379    9    1419  441    15       0  29.598127 -37.200936  453   
14380     14380    9    1420  441    15       0  29.598127 -37.200936  453   
14381     14381    9    1421  441    15       0  29.598127 -37.200936  453   
14382     14382    9    1422  441    15       0  29.598127 -37.200936  453   
14383     14383    9    1423  441    15       0  29.598127 -37.200936  453   
14384     14384    9    1424  441    15       0  29.598127 -37.200936  453   
14385     14385    9    1425  441    15       0  29.598127 -37.200936  453   
14386     14386    9    1426  441    15       0  29.598127 -37.200936  453   
14387     14387    9    1427  441    15       0  29.598127 -37.200936  453   
14388     14388    9    1428  441    15       0  29.598127 -37.200936  453   
14389     14389    9    1429 